<a href="https://colab.research.google.com/github/m-tari/arxiv_interface/blob/master/notebooks/05_classification_with_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets pandas

     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 306 kB 43.9 MB/s 
     |████████████████████████████████| 3.3 MB 25.2 MB/s 
     |████████████████████████████████| 596 kB 51.9 MB/s 
     |████████████████████████████████| 61 kB 518 kB/s 
     |████████████████████████████████| 895 kB 52.5 MB/s 
     |████████████████████████████████| 1.1 MB 36.9 MB/s 
     |████████████████████████████████| 243 kB 56.5 MB/s 
     |████████████████████████████████| 132 kB 56.2 MB/s 
     |████████████████████████████████| 271 kB 53.4 MB/s 
     |████████████████████████████████| 160 kB 51.2 MB/s 
     |████████████████████████████████| 192 kB 57.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Load dataset

In [ ]:
from datasets import Dataset, load_metric, load_dataset
import pandas as pd

In [ ]:
sample_df = pd.read_csv("/content/drive/MyDrive/ML/sample_df_2021.csv", nrows=10000, usecols=['abstract', 'general_category'])

In [ ]:
sample_df

,abstract,general_category
0,Impact evaluation in public engagement neces...,"['physics', 'physics']"
1,The direct variational optimization of the t...,['physics']
2,"In a graph $G$, a subset of vertices is a di...",['math']
3,How to form groups in a mobility system that...,"['cs', 'math']"
4,In this manuscript we propose a theoretical ...,['physics']
...,...,...
9995,The objective of this work was to know the a...,"['econ', 'q-fin']"
9996,A small fraction of thermalized dark radiati...,"['astro-ph', 'astro-ph', 'hep-ph']"
9997,We study decay properties for solutions to t...,['math']
9998,The process of the excitation of an electrom...,"['physics', 'physics']"


In [ ]:
sample_df['labels'] = sample_df.general_category.str.replace("[\[\]\']", "",).str.split(pat=", ")


In [ ]:
sample_df['text'] = sample_df['abstract']

In [ ]:
sample_df

,abstract,general_category,labels,text
0,Impact evaluation in public engagement neces...,"['physics', 'physics']","[physics, physics]",Impact evaluation in public engagement neces...
1,The direct variational optimization of the t...,['physics'],[physics],The direct variational optimization of the t...
2,"In a graph $G$, a subset of vertices is a di...",['math'],[math],"In a graph $G$, a subset of vertices is a di..."
3,How to form groups in a mobility system that...,"['cs', 'math']","[cs, math]",How to form groups in a mobility system that...
4,In this manuscript we propose a theoretical ...,['physics'],[physics],In this manuscript we propose a theoretical ...
...,...,...,...,...
9995,The objective of this work was to know the a...,"['econ', 'q-fin']","[econ, q-fin]",The objective of this work was to know the a...
9996,A small fraction of thermalized dark radiati...,"['astro-ph', 'astro-ph', 'hep-ph']","[astro-ph, astro-ph, hep-ph]",A small fraction of thermalized dark radiati...
9997,We study decay properties for solutions to t...,['math'],[math],We study decay properties for solutions to t...
9998,The process of the excitation of an electrom...,"['physics', 'physics']","[physics, physics]",The process of the excitation of an electrom...


In [ ]:
sample_df = sample_df[['text', 'labels']][:10000]

In [ ]:
sample_df.labels = sample_df.labels.apply(set).apply(list)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(sample_df.labels)
y_train = pd.DataFrame(y, columns=multilabel.classes_)

In [ ]:
y_train

,astro-ph,cond-mat,cs,econ,eess,gr-qc,hep-ex,hep-lat,hep-ph,hep-th,math,math-ph,nlin,nucl-ex,nucl-th,physics,q-bio,q-fin,quant-ph,stat
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9996,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
s = pd.Series([enc_label for enc_label in y])

In [ ]:
s

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
3       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
9995    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9996    [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
9997    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
9998    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9999    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...
Length: 10000, dtype: object

In [ ]:
sample_df['enc_labels'] = s

In [ ]:
sample_df.enc_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [ ]:
import numpy as np
y_pred = np.array([[1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

In [ ]:
y_pred.shape

(1, 20)

In [ ]:
multilabel.inverse_transform(y_pred)

[('astro-ph', 'cond-mat')]

In [ ]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
sample_dataset = Dataset.from_pandas(sample_df[['text', 'enc_labels']])

In [ ]:
sample_dataset = sample_dataset.train_test_split(test_size=0.1)

In [ ]:
sample_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'enc_labels'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['text', 'enc_labels'],
        num_rows: 1000
    })
})

In [ ]:
sample_dataset['train'][1]

{'enc_labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 'text': '  The estimation of the density operator that describe the state of a quantum\nsystem, when an informational complete measurement is not available, can be\nperformed, in a reliable way, by adopting the Maximum Entropy principle\n(MaxEnt) as additional criterion. In this paper, we have studied the efficiency\nof the MaxEnt method for quantum states estimation when there is prior\ninformation about symmetries of the state. We explicitly describe how to\nimplement the algorithm, in the most general case, and implemented it to\nconduct numerical simulations estimating the density matrix of several\nthree-qubit quantum state of interest. Furthermore, we analyze the performance\nof the method in realistic quantum information scenarios. We observe that, for\nmost states, this approach allows to considerably reduce the number of\nindependent measurements needed to obtain a sufficiently high fidelity, and it\

## Load DistilBERT model

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

model_name = "distilbert-base-uncased"
num_labels = 20

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = (AutoModelForSequenceClassification.from_pretrained(model_name, problem_type="multi_label_classification", num_labels=num_labels).to(device))

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19"
  },
  "initializer_range": 0.02,
  "label2id"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, problem_type="multi_label_classification")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

## Tokenizing

In [ ]:
def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
# remove all columns except for enc_labels
cols = sample_dataset["train"].column_names
cols.remove("enc_labels")

sample_dataset_encoded = sample_dataset.map(tokenize, batched=True, batch_size=None, remove_columns=cols)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
sample_dataset_encoded['train']

Dataset({
    features: ['attention_mask', 'enc_labels', 'input_ids'],
    num_rows: 9000
})

In [ ]:
# cast label IDs to floats
sample_dataset_encoded.set_format("torch")

In [ ]:
sample_dataset_encoded['train']

Dataset({
    features: ['attention_mask', 'enc_labels', 'input_ids'],
    num_rows: 9000
})

In [ ]:
sample_dataset_encoded = (sample_dataset_encoded
          .map(lambda x : {"float_labels": x["enc_labels"].to(torch.float)})
          .rename_column("float_labels", "labels"))

  0%|          | 0/9000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [ ]:
sample_dataset_encoded['train'][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0

## Training

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(".", num_train_epochs=1)

trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer,
    train_dataset=sample_dataset_encoded["train"],
    eval_dataset=sample_dataset_encoded["test"])

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: enc_labels.
***** Running training *****
  Num examples = 9000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Step,Training Loss
500,0.165700
1000,0.106100


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=0.13158036126030817, metrics={'train_runtime': 887.8355, 'train_samples_per_second': 10.137, 'train_steps_per_second': 1.267, 'total_flos': 1192589291520000.0, 'train_loss': 0.13158036126030817, 'epoch': 1.0})

In [ ]:
!pwd

/content


In [ ]:
pt_save_directory = '/content/drive/MyDrive/ML/'
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)

tokenizer config file saved in /content/drive/MyDrive/ML/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ML/special_tokens_map.json
Configuration saved in /content/drive/MyDrive/ML/config.json
Model weights saved in /content/drive/MyDrive/ML/pytorch_model.bin


## Inference

In [ ]:
# DistilBert uses a max_length of 512 so we cut the article to 512 tokens.

abstract = """
Data scientists and statisticians are often at odds when determining the best approach, machine learning or statistical modeling, 
to solve an analytics challenge. However, machine learning and statistical modeling are more cousins than adversaries on different 
sides of an analysis battleground. Choosing between the two approaches or in some cases using both is based on the problem to be solved 
and outcomes required as well as the data available for use and circumstances of the analysis. Machine learning and statistical modeling 
are complementary, based on similar mathematical principles, but simply using different tools in an overall analytics knowledge base. 
Determining the predominant approach should be based on the problem to be solved as well as empirical evidence, such as size and completeness 
of the data, number of variables, assumptions or lack thereof, and expected outcomes such as predictions or causality. Good analysts and 
data scientists should be well versed in both techniques and their proper application, thereby using the right tool for the right project 
to achieve the desired results. """

actual_category = "cs, stat"  

inputs = tokenizer(abstract, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

In [ ]:
outputs = model(**inputs)

In [ ]:
outputs

SequenceClassifierOutput([('logits',
                           tensor([[-4.0147, -3.4506,  0.2246, -2.7267, -2.5622, -4.6152, -4.5388, -4.7027,
                                    -4.5119, -5.1111, -2.3533, -4.8904, -4.1490, -4.6337, -4.8049, -3.0671,
                                    -4.0729, -3.3210, -3.8664,  0.9501]], device='cuda:0',
                                  grad_fn=<AddmmBackward0>))])

In [ ]:
# Because we can have multiple lables for an abstract and they are NOT mutually exclusive, we pass logits to sigmoid function
probs = torch.sigmoid(outputs.logits).tolist()[0]

In [ ]:
preds = [1 for prob in probs if prob>0.5 else 0]

[0.01772821694612503,
 0.030749831348657608,
 0.5559039115905762,
 0.06141582876443863,
 0.07160800695419312,
 0.009803163819015026,
 0.010573271661996841,
 0.008989191614091396,
 0.01085792388767004,
 0.005993204191327095,
 0.0868067815899849,
 0.007462218403816223,
 0.015535667538642883,
 0.009624989703297615,
 0.008122998289763927,
 0.04448676481842995,
 0.016743015497922897,
 0.03485780209302902,
 0.020504316315054893,
 0.721133291721344]

In [ ]:
categories = multilabel.inverse_transform(preds)

print(f'Actual Categories: {actual_category}\n')
print(f'Predicted Categories: {categories}\n')

In [ ]:
# predicted cs and stat, which are above 0.50 threshold! :)

## Error Analysis